# インシデント調査

M先輩から「このサーバがちょっと怪しいくって調査したいから、[この本](https://www.amazon.co.jp/dp/4822279871/)のp179に載ってるサーバの調査コマンド一式の結果のレポート作って。あ、あと、ポートが1つ LISTEN になってて http サーバになってるはずなんで、それが動いてるかも確認しといて。」と指示があった。

# ライブレスポンス実施

インシデントレスポンス第3版p179に従って作業を行う。

## システムの日付と時間

In [1]:
!date

Thu May 31 09:29:03 UTC 2018


あ、UTCだ。JSTにしないと。このサーバ、[このコンテナ](https://hub.docker.com/r/mnagaku/lc-minimal-notebook/)で、環境変数でTZ変えれるっぽい。

In [2]:
%env TZ JST-9

env: TZ=JST-9


In [3]:
!date

Thu May 31 18:33:23 JST 2018


JSTになった。

## インストールされているソフトウェア

In [4]:
!dpkg --get-selections

adduser						install
apt						install
base-files					install
base-passwd					install
bash						install
bsdutils					install
bzip2						install
ca-certificates					install
coreutils					install
curl						install
dash						install
debconf						install
debianutils					install
diffutils					install
dpkg						install
e2fslibs:amd64					install
e2fsprogs					install
findutils					install
fonts-ipafont-gothic				install
fonts-ipafont-mincho				install
fonts-liberation				install
gcc-5-base:amd64				install
gcc-6-base:amd64				install
git						install
git-man						install
gnupg						install
gpgv						install
grep						install
gzip						install
hostname					install
init						install
init-system-helpers				install
initscripts					install
insserv						install
iproute2					install
libacl1:amd64					install
libapparmor1:amd64				install
libapt-pkg5.0:amd64				install
libasn1-8-heimdal:amd64				install
libattr1:amd64					install
libaudit-common					install
libaudit1:amd64					install
libblkid1:amd64

## ファイルシステム情報

In [5]:
!mount -l

overlay on / type overlay (rw,relatime,lowerdir=/var/lib/docker/overlay2/l/DKAYFZOCBEW4GYDL6ND5JW3355:/var/lib/docker/overlay2/l/6RZM5RHM4BHP44T5KUIBKYEYRD:/var/lib/docker/overlay2/l/2LHW5XITY6FEN5G4KCPXLLZ74N:/var/lib/docker/overlay2/l/MVRYHXHPQKHWTYQ4HDAHLF5ZYB:/var/lib/docker/overlay2/l/JCC4VJO7YUOXS5QIVKLSFAZT2D:/var/lib/docker/overlay2/l/YYLYZIVG3RGO6W63UK322OWJZ2:/var/lib/docker/overlay2/l/R7SDMTKLXEZ7NU72FGGOYNC52G:/var/lib/docker/overlay2/l/WR74YQX5QWH3NDCC2LKXYOZKT4:/var/lib/docker/overlay2/l/YTIAAJP6BNXMMWACQBYOSF3MQJ:/var/lib/docker/overlay2/l/M5WAHCB4H5JRNJDRXMR4UW2QUF:/var/lib/docker/overlay2/l/SJZUPMHEUYSMYJM3HTTHWDIAPE:/var/lib/docker/overlay2/l/AZ7K7NHFGSGACBMXNSRTTRZ2YR:/var/lib/docker/overlay2/l/LOXXHLOLUW2O3ANVMUMVSADQYQ:/var/lib/docker/overlay2/l/HUG5F3HE2LKFETLTSEK2BANKI2:/var/lib/docker/overlay2/l/Q674H2GONO5744GUV6I7EQ35NI:/var/lib/docker/overlay2/l/NQMMGA364YW7J3EJLE3NB435UQ:/var/lib/docker/overlay2/l/4RZINMFLPUFUNS76ACRJAEKU6N:/var/lib/docker/overlay2/l/ETKFDFE

In [6]:
!df -Th

Filesystem     Type     Size  Used Avail Use% Mounted on
overlay        overlay   59G  4.9G   51G   9% /
tmpfs          tmpfs     64M     0   64M   0% /dev
tmpfs          tmpfs    991M     0  991M   0% /sys/fs/cgroup
/dev/sda1      ext4      59G  4.9G   51G   9% /etc/hosts
shm            tmpfs     64M     0   64M   0% /dev/shm
tmpfs          tmpfs    991M     0  991M   0% /sys/firmware


In [7]:
!fdisk -l

## OSのバージョン

In [8]:
!cat /etc/issue

Ubuntu 16.04.4 LTS \n \l



## カーネルのバージョン

In [9]:
!uname -a

Linux 8f91f8204652 4.9.87-linuxkit-aufs #1 SMP Wed Mar 14 15:12:16 UTC 2018 x86_64 x86_64 x86_64 GNU/Linux


## 稼働時間

In [10]:
!w

 18:38:55 up  5:40,  0 users,  load average: 0.02, 0.02, 0.00
USER     TTY      FROM             LOGIN@   IDLE   JCPU   PCPU WHAT


## cron

In [11]:
!ls -la /var/spool/cron

ls: cannot access '/var/spool/cron': No such file or directory


RuntimeError: Unexpected exit code: 2

/var/spool/cron が無い？（凍結して再実行スキップにしておく）

In [12]:
!sudo ls -la /var/spool/cron

ls: cannot access '/var/spool/cron': No such file or directory


RuntimeError: Unexpected exit code: 2

sudo でも見えないので、無いんだろう。無いから tar できない。

In [13]:
## サービス

間違えた

## サービス

本には具体的なコマンドの指定がないが、ubuntuなので、sysv-rc-confを使う。[参考情報](https://www.laddy.info/2017/11/28793/)

In [14]:
!sudo sysv-rc-conf --list

sudo: sysv-rc-conf: command not found


RuntimeError: Unexpected exit code: 1

入ってないので入れる。

In [15]:
!apt-get update
!apt-get install -y sysv-rc-conf


E: List directory /var/lib/apt/lists/partial is missing. - Acquire (13: Permission denied)


RuntimeError: Unexpected exit code: 100

ああ、sudo...

In [16]:
!sudo apt-get update
!sudo apt-get install -y sysv-rc-conf

Get:1 http://archive.ubuntu.com/ubuntu xenial InRelease [247 kB]
Get:2 http://security.ubuntu.com/ubuntu xenial-security InRelease [107 kB]
Get:3 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [109 kB]
Get:4 http://security.ubuntu.com/ubuntu xenial-security/universe Sources [80.9 kB]
Get:5 http://archive.ubuntu.com/ubuntu xenial-backports InRelease [107 kB]     
Get:6 http://archive.ubuntu.com/ubuntu xenial/universe Sources [9,802 kB]
Get:7 http://security.ubuntu.com/ubuntu xenial-security/main amd64 Packages [637 kB]
Get:8 http://archive.ubuntu.com/ubuntu xenial/main amd64 Packages [1,558 kB]  
Get:9 http://archive.ubuntu.com/ubuntu xenial/restricted amd64 Packages [14.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu xenial/universe amd64 Packages [9,827 kB]
Get:11 http://archive.ubuntu.com/ubuntu xenial/multiverse amd64 Packages [176 kB]
Get:12 http://security.ubuntu.com/ubuntu xenial-security/restricted amd64 Packages [12.7 kB]
Get:13 http://archive.ubuntu.com/ubuntu xenial-

In [17]:
!sudo sysv-rc-conf --list

halt         0:off
killprocs    1:on
ondemand     2:on	3:on	4:on	5:on
procps       S:on
rc.local     2:on	3:on	4:on	5:on
reboot       6:off
sendsigs     0:off	6:off
single       1:on
umountfs     0:off	6:off
umountroot   0:off	6:off
urandom      0:off	6:off	S:on


## ユーザアカウント

In [18]:
!cat /etc/passwd

root:x:0:0:root:/root:/bin/bash
daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
bin:x:2:2:bin:/bin:/usr/sbin/nologin
sys:x:3:3:sys:/dev:/usr/sbin/nologin
sync:x:4:65534:sync:/bin:/bin/sync
games:x:5:60:games:/usr/games:/usr/sbin/nologin
man:x:6:12:man:/var/cache/man:/usr/sbin/nologin
lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin
mail:x:8:8:mail:/var/mail:/usr/sbin/nologin
news:x:9:9:news:/var/spool/news:/usr/sbin/nologin
uucp:x:10:10:uucp:/var/spool/uucp:/usr/sbin/nologin
proxy:x:13:13:proxy:/bin:/usr/sbin/nologin
www-data:x:33:33:www-data:/var/www:/usr/sbin/nologin
backup:x:34:34:backup:/var/backups:/usr/sbin/nologin
list:x:38:38:Mailing List Manager:/var/list:/usr/sbin/nologin
irc:x:39:39:ircd:/var/run/ircd:/usr/sbin/nologin
gnats:x:41:41:Gnats Bug-Reporting System (admin):/var/lib/gnats:/usr/sbin/nologin
nobody:x:65534:65534:nobody:/nonexistent:/usr/sbin/nologin
systemd-timesync:x:100:102:systemd Time Synchronization,,,:/run/systemd:/bin/false
systemd-network:x:101:103:systemd Networ

In [19]:
!sudo cat /etc/shadow

root:*:17590:0:99999:7:::
daemon:*:17590:0:99999:7:::
bin:*:17590:0:99999:7:::
sys:*:17590:0:99999:7:::
sync:*:17590:0:99999:7:::
games:*:17590:0:99999:7:::
man:*:17590:0:99999:7:::
lp:*:17590:0:99999:7:::
mail:*:17590:0:99999:7:::
news:*:17590:0:99999:7:::
uucp:*:17590:0:99999:7:::
proxy:*:17590:0:99999:7:::
www-data:*:17590:0:99999:7:::
backup:*:17590:0:99999:7:::
list:*:17590:0:99999:7:::
irc:*:17590:0:99999:7:::
gnats:*:17590:0:99999:7:::
nobody:*:17590:0:99999:7:::
systemd-timesync:*:17590:0:99999:7:::
systemd-network:*:17590:0:99999:7:::
systemd-resolve:*:17590:0:99999:7:::
systemd-bus-proxy:*:17590:0:99999:7:::
_apt:*:17590:0:99999:7:::
jovyan:!:17616:0:99999:7:::


## グループ

In [20]:
!cat /etc/group

root:x:0:
daemon:x:1:
bin:x:2:
sys:x:3:
adm:x:4:
tty:x:5:
disk:x:6:
lp:x:7:
mail:x:8:
news:x:9:
uucp:x:10:
man:x:12:
proxy:x:13:
kmem:x:15:
dialout:x:20:
fax:x:21:
voice:x:22:
cdrom:x:24:
floppy:x:25:
tape:x:26:
sudo:x:27:
audio:x:29:
dip:x:30:
www-data:x:33:
backup:x:34:
operator:x:37:
list:x:38:
irc:x:39:
src:x:40:
gnats:x:41:
shadow:x:42:
utmp:x:43:
video:x:44:
sasl:x:45:
plugdev:x:46:
staff:x:50:
games:x:60:
users:x:100:
nogroup:x:65534:
systemd-journal:x:101:
systemd-timesync:x:102:
systemd-network:x:103:
systemd-resolve:x:104:
systemd-bus-proxy:x:105:


## ネットワークインタフェース

今なら、ipコマンドだよな。

In [21]:
!ip a

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
2: tunl0@NONE: <NOARP> mtu 1480 qdisc noop state DOWN group default qlen 1
    link/ipip 0.0.0.0 brd 0.0.0.0
3: ip6tnl0@NONE: <NOARP> mtu 1452 qdisc noop state DOWN group default qlen 1
    link/tunnel6 :: brd ::
9: eth0@if10: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:ac:11:00:02 brd ff:ff:ff:ff:ff:ff link-netnsid 0
    inet 172.17.0.2/16 brd 172.17.255.255 scope global eth0
       valid_lft forever preferred_lft forever


In [22]:
!ip -s l

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN mode DEFAULT group default qlen 1
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    RX: bytes  packets  errors  dropped overrun mcast   
    2326626    33140    0       0       0       0       
    TX: bytes  packets  errors  dropped carrier collsns 
    2326626    33140    0       0       0       0       
2: tunl0@NONE: <NOARP> mtu 1480 qdisc noop state DOWN mode DEFAULT group default qlen 1
    link/ipip 0.0.0.0 brd 0.0.0.0
    RX: bytes  packets  errors  dropped overrun mcast   
    0          0        0       0       0       0       
    TX: bytes  packets  errors  dropped carrier collsns 
    0          0        0       0       0       0       
3: ip6tnl0@NONE: <NOARP> mtu 1452 qdisc noop state DOWN mode DEFAULT group default qlen 1
    link/tunnel6 :: brd ::
    RX: bytes  packets  errors  dropped overrun mcast   
    0          0        0       0       0       0       
    TX: bytes  packets  errors 

## ルーティングテーブル

In [23]:
!ip r

default via 172.17.0.1 dev eth0 
172.17.0.0/16 dev eth0  proto kernel  scope link  src 172.17.0.2 


## ARPテーブル

In [24]:
!ip n

172.17.0.1 dev eth0 lladdr 02:42:26:0f:20:35 REACHABLE


## ネットワーク接続

TCP、UDPそれぞれssコマンドで見ることにする。

In [25]:
!ss -nat

State      Recv-Q Send-Q Local Address:Port               Peer Address:Port              
LISTEN     0      100    127.0.0.1:47481                    *:*                  
LISTEN     0      100    127.0.0.1:55325                    *:*                  
LISTEN     0      100    127.0.0.1:40127                    *:*                  
LISTEN     0      100    127.0.0.1:59487                    *:*                  
LISTEN     0      100    127.0.0.1:51399                    *:*                  
LISTEN     0      100    127.0.0.1:37991                    *:*                  
LISTEN     0      100    127.0.0.1:60393                    *:*                  
LISTEN     0      100    127.0.0.1:56715                    *:*                  
LISTEN     0      100    127.0.0.1:46347                    *:*                  
LISTEN     0      100    127.0.0.1:36331                    *:*                  
LISTEN     0      100    127.0.0.1:52781                    *:*                  
LISTEN  

In [26]:
!ss -nau

State      Recv-Q Send-Q Local Address:Port               Peer Address:Port              


## ロードされているドライバ

In [27]:
!lsmod

/bin/sh: 1: lsmod: not found


RuntimeError: Unexpected exit code: 127

In [28]:
!sudo apt-get install -y lsmod

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package lsmod


RuntimeError: Unexpected exit code: 100

ああ、lsmodは、kmodをインスコするんだった。[参考情報](https://stackoverflow.com/questions/38229579/docker-container-lsmod-not-found)

In [29]:
!sudo apt-get install -y kmod

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  kmod
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 88.6 kB of archives.
After this operation, 243 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 kmod amd64 22-1ubuntu5 [88.6 kB]
Fetched 88.6 kB in 2s (34.7 kB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package kmod.
(Reading database ... 11485 files and directories currently installed.)
Preparing to unpack .../kmod_22-1ubuntu5_amd64.deb ...
Unpacking kmod (22-1ubuntu5) ...
Processing triggers for systemd (229-4ubuntu21.2) ...
Setting up kmod (22-1ubuntu5) ...
Processing triggers for systemd (229-4ubuntu21.2) ...


In [30]:
!lsmod

Module                  Size  Used by
xfrm_user              32768  1
xfrm_algo              16384  1 xfrm_user
tpm_tis                16384  0
tpm_tis_core           20480  1 tpm_tis
tpm                    32768  2 tpm_tis,tpm_tis_core


## オープン中のファイルとハンドル

In [31]:
!lsof

/bin/sh: 1: lsof: not found


RuntimeError: Unexpected exit code: 127

In [32]:
!sudo apt-get install -y lsof

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  lsof
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 248 kB of archives.
After this operation, 451 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu xenial/main amd64 lsof amd64 4.89+dfsg-0.1 [248 kB]
Fetched 248 kB in 4s (50.7 kB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package lsof.
(Reading database ... 11528 files and directories currently installed.)
Preparing to unpack .../lsof_4.89+dfsg-0.1_amd64.deb ...
Unpacking lsof (4.89+dfsg-0.1) ...
Setting up lsof (4.89+dfsg-0.1) ...


In [33]:
!lsof

COMMAND   PID TID   USER   FD      TYPE             DEVICE SIZE/OFF    NODE NAME
tini        1     jovyan  cwd       DIR               0,59     4096   20326 /home/jovyan
tini        1     jovyan  rtd       DIR               0,59     4096   20290 /
tini        1     jovyan  txt       REG                8,1    19864  656083 /usr/local/bin/tini
tini        1     jovyan  mem       REG                8,1  1868984  394365 /lib/x86_64-linux-gnu/libc-2.23.so
tini        1     jovyan  mem       REG                8,1   162632  394345 /lib/x86_64-linux-gnu/ld-2.23.so
tini        1     jovyan    0r     FIFO               0,10      0t0   20336 pipe
tini        1     jovyan    1w     FIFO               0,10      0t0   20337 pipe
tini        1     jovyan    2w     FIFO               0,10      0t0   20338 pipe
jupyter-n   5     jovyan  cwd       DIR               0,59     4096   20326 /home/jovyan
jupyter-n   5     jovyan  rtd       DIR               0,59     4096   20290 /
jupyter-n   5     jovyan  

ZMQbg/0   103 109 jovyan  mem       REG                8,1   963624  530898 /opt/conda/lib/python3.6/lib-dynload/unicodedata.cpython-36m-x86_64-linux-gnu.so
ZMQbg/0   103 109 jovyan  mem       REG                8,1   187746  530880 /opt/conda/lib/python3.6/lib-dynload/array.cpython-36m-x86_64-linux-gnu.so
ZMQbg/0   103 109 jovyan  mem       REG                8,1   249585  529581 /opt/conda/lib/libtinfo.so.5.9
ZMQbg/0   103 109 jovyan  mem       REG                8,1   176142  529542 /opt/conda/lib/libncurses.so.5.9
ZMQbg/0   103 109 jovyan  mem       REG                8,1  1165114  529569 /opt/conda/lib/libsqlite3.so.0.8.6
ZMQbg/0   103 109 jovyan  mem       REG                8,1   295030  530872 /opt/conda/lib/python3.6/lib-dynload/_sqlite3.cpython-36m-x86_64-linux-gnu.so
ZMQbg/0   103 109 jovyan  mem       REG                8,1   433613  530864 /opt/conda/lib/python3.6/lib-dynload/_pickle.cpython-36m-x86_64-linux-gnu.so
ZMQbg/0   103 109 jovyan  mem       REG                8,1

## 実行中のプロセスとスレッド

In [34]:
!ps auxwwwem

USER        PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
jovyan        1  0.0  0.0   4360   628 ?        -    16:04   0:00 tini -- jupyter notebook PATH=/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin HOSTNAME=8f91f8204652 DEBIAN_FRONTEND=noninteractive CONDA_DIR=/opt/conda SHELL=/bin/bash NB_USER=jovyan NB_UID=1000 NB_GID=100 LC_ALL=en_US.UTF-8 LANG=en_US.UTF-8 LANGUAGE=en_US.UTF-8 HOME=/home/jovyan MINICONDA_VERSION=4.5.1 BASH_ENV=/etc/bash_env
jovyan        -  0.0    -      -     - -        Ss   16:04   0:00 -
jovyan        5  0.0  3.1 368528 63312 ?        -    16:04   0:09 /opt/conda/bin/python /opt/conda/bin/jupyter-notebook PATH=/opt/conda/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin HOSTNAME=8f91f8204652 DEBIAN_FRONTEND=noninteractive CONDA_DIR=/opt/conda SHELL=/bin/bash NB_USER=jovyan NB_UID=1000 NB_GID=100 LC_ALL=en_US.UTF-8 LANG=en_US.UTF-8 LANGUAGE=en_US.UTF-8 HOME=/home/jovyan MINICONDA_VERSION

## 設定情報の保全

In [35]:
!rm -f host_etc.tar.gz ; sudo tar cvzf host_etc.tar.gz /etc/ ; ls -la host_etc.tar.gz

tar: Removing leading `/' from member names
/etc/
/etc/passwd
/etc/rc1.d/
/etc/rc1.d/S02single
/etc/rc1.d/README
/etc/rc1.d/S01killprocs
/etc/cron.weekly/
/etc/cron.weekly/fstrim
/etc/selinux/
/etc/selinux/semanage.conf
/etc/rc.local
/etc/update-motd.d/
/etc/update-motd.d/00-header
/etc/update-motd.d/10-help-text
/etc/dhcp/
/etc/dhcp/dhclient-exit-hooks.d/
/etc/dhcp/dhclient-exit-hooks.d/timesyncd
/etc/rc6.d/
/etc/rc6.d/K03umountfs
/etc/rc6.d/K02umountnfs.sh
/etc/rc6.d/K05reboot
/etc/rc6.d/K01sendsigs
/etc/rc6.d/README
/etc/rc6.d/K01hwclock.sh
/etc/rc6.d/K01urandom
/etc/rc6.d/K04umountroot
/etc/sysctl.d/
/etc/sysctl.d/10-console-messages.conf
/etc/sysctl.d/10-link-restrictions.conf
/etc/sysctl.d/10-network-security.conf
/etc/sysctl.d/10-magic-sysrq.conf
/etc/sysctl.d/10-ipv6-privacy.conf
/etc/sysctl.d/10-ptrace.conf
/etc/sysctl.d/README
/etc/sysctl.d/10-zeropage.conf
/etc/sysctl.d/99-sysctl.conf
/etc/sysctl.d/10-kernel-hardening.conf
/etc/init.d/
/etc/init.d/checkroot-bootclean.sh
/etc

/etc/mailcap.order
/etc/ssl/
/etc/ssl/certs/
/etc/ssl/certs/861a399d.0
/etc/ssl/certs/b7db1890.0
/etc/ssl/certs/Certum_Root_CA.pem
/etc/ssl/certs/cd8c0d63.0
/etc/ssl/certs/244b5494.0
/etc/ssl/certs/f060240e.0
/etc/ssl/certs/062cdee6.0
/etc/ssl/certs/d4dae3dd.0
/etc/ssl/certs/VeriSign_Class_3_Public_Primary_Certification_Authority_-_G4.pem
/etc/ssl/certs/5f47b495.0
/etc/ssl/certs/080911ac.0
/etc/ssl/certs/LuxTrust_Global_Root_2.pem
/etc/ssl/certs/bd1910d4.0
/etc/ssl/certs/87753b0d.0
/etc/ssl/certs/e113c810.0
/etc/ssl/certs/cc450945.0
/etc/ssl/certs/034868d6.0
/etc/ssl/certs/ba89ed3b.0
/etc/ssl/certs/02265526.0
/etc/ssl/certs/DigiCert_Global_Root_G2.pem
/etc/ssl/certs/Hongkong_Post_Root_CA_1.pem
/etc/ssl/certs/Certum_Trusted_Network_CA.pem
/etc/ssl/certs/5cd81ad7.0
/etc/ssl/certs/ccc52f49.0
/etc/ssl/certs/399e7759.0
/etc/ssl/certs/Amazon_Root_CA_3.pem
/etc/ssl/certs/f3377b1b.0
/etc/ssl/certs/3c860d51.0
/etc/ssl/certs/40193066.0
/etc/ssl/certs/8b59b1ad.0
/etc/ssl/certs/D-TRUST_Root_Class_

/etc/perl/Net/
/etc/perl/Net/libnet.cfg
/etc/perl/CPAN/
/etc/iproute2/
/etc/iproute2/rt_protos
/etc/iproute2/rt_dsfield
/etc/iproute2/rt_realms
/etc/iproute2/rt_tables
/etc/iproute2/rt_scopes
/etc/iproute2/nl_protos
/etc/iproute2/group
/etc/iproute2/ematch_map
/etc/vim/
/etc/vim/vimrc
/etc/gss/
/etc/gss/mech.d/
/etc/gss/mech.d/README
/etc/jupyter/
/etc/jupyter/jupyter_notebook_config.py
/etc/subgid-
/etc/shadow-
/etc/subuid-
/etc/passwd-
/etc/locale.gen
/etc/ca-certificates.conf
/etc/wgetrc
/etc/locale.alias
/etc/ca-certificates/
/etc/ca-certificates/update.d/
-rw-r--r-- 1 root users 262641 May 31 19:33 host_etc.tar.gz


## システムログの保全

In [36]:
!rm -f host_log.tar.gz ; sudo tar cvzf host_log.tar.gz /var/log/ ; ls -la host_log.tar.gz

tar: Removing leading `/' from member names
/var/log/
/var/log/dmesg
/var/log/bootstrap.log
/var/log/fsck/
/var/log/fsck/checkroot
/var/log/fsck/checkfs
/var/log/faillog
/var/log/dpkg.log
/var/log/lastlog
/var/log/apt/
/var/log/apt/history.log
/var/log/apt/term.log
/var/log/alternatives.log
/var/log/btmp
/var/log/wtmp
-rw-r--r-- 1 root users 25598 May 31 19:33 host_log.tar.gz


## ユーザのシェル履歴

In [37]:
!history

/bin/sh: 1: history: not found


RuntimeError: Unexpected exit code: 127

M先輩と相談した結果、コンテナの特性上、ここではhistory取れないっぽいので、ターミナルで取ってみる、

取って来たのを貼っておく。
```
jovyan@8f91f8204652:~$ history
    1  history
    2  sudo -i
    3  exit
    4  history
    5  ls
    6  history
    7  sudo -i
    8  exit
    9  history
jovyan@8f91f8204652:~$ sudo -i
root@8f91f8204652:~# history
    1  history
    2  exit
    3  history
    4  ls
    5  exit
    6  history
root@8f91f8204652:~# exit
logout
jovyan@8f91f8204652:~$ exit
exit
```

## ファイル一覧

In [38]:
!rm -f host_files.txt ; sudo find / -xdev -printf "%m;%Ax;%AT;%Tx;%TT;%Cx;%CT;%U;%G;%s;%p;\n" > host_files.txt
!ls -la host_files.txt

-rw-r--r-- 1 jovyan users 13730667 May 31 19:39 host_files.txt


## ライブレスポンス出力チェックサム計算

ファイルを、host_etc.tar.gz、host_files.txt、host_log.tar.gzと3つ作ったので、md5sumしておく。

In [39]:
!md5sum host_etc.tar.gz

66200cc0c26a53f732f82d10351bc7f3  host_etc.tar.gz


In [40]:
!md5sum host_files.txt

cb541008a1d2f521b58e0b989fc9b8f9  host_files.txt


In [41]:
!md5sum host_log.tar.gz

947f284a5d7ab99aa9bbeb5086c343b6  host_log.tar.gz


# httpサーバの動確

LISTENポートを探して、httpでリクエスト出して、振る舞いを確認しておく。

まず、LISTENポートについては、[ネットワーク接続](#ネットワーク接続)で8888だけ全てのネットワークインタフェースに対してLISTENしていることが分かっている。外部に対して繋がっているネットワークインタフェースは[ルーティングテーブル](#ルーティングテーブル)と[ネットワークインタフェース](#ネットワークインタフェース)からeth0の172.17.0.2だと分かるので、ここにwgetしてみる。

In [42]:
!wget -O - http://172.17.0.2:8888

--2018-05-31 20:21:18--  http://172.17.0.2:8888/
Connecting to 172.17.0.2:8888... connected.
HTTP request sent, awaiting response... 302 Found
Location: /tree? [following]
--2018-05-31 20:21:18--  http://172.17.0.2:8888/tree?
Reusing existing connection to 172.17.0.2:8888.
HTTP request sent, awaiting response... 302 Found
Location: /login?next=%2Ftree%3F [following]
--2018-05-31 20:21:18--  http://172.17.0.2:8888/login?next=%2Ftree%3F
Reusing existing connection to 172.17.0.2:8888.
HTTP request sent, awaiting response... 200 OK
Length: 8818 (8.6K) [text/html]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               <!DOCTYPE HTML>
<html>

<head>
    <meta charset="utf-8">

    <title>Jupyter Notebook</title>
    <link id="favicon" rel="shortcut icon" type="image/x-icon" href="/static/base/images/favicon.ico?v=97c6417ed01bdc0ae3ef32ae4894fd03">
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    <link rel="stylesheet" href="/static/

302Foundで2回飛ばされて、ログインページに行って、htmlをもらえた。動いているのを確認できた。

# おしまい